In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
df = pd.read_csv(r'all_drinks.csv', delimiter = ",")

In [3]:
def to_lower(text):
    return text.lower()


df['strGlass'] = df['strGlass'].apply(to_lower)
df['strCategory'] = df['strCategory'].apply(to_lower)

In [4]:
conn = sqlite3.connect('cocktails_database.db')

In [5]:
df_glass = pd.read_sql_query("SELECT * FROM glass", conn)

In [6]:
df = pd.merge(df, df_glass, left_on='strGlass', right_on='glass')

In [7]:
df.columns

Index(['Unnamed: 0', 'strDrink', 'dateModified', 'idDrink', 'strAlcoholic',
       'strCategory', 'strDrinkThumb', 'strGlass', 'strIBA', 'strIngredient1',
       'strIngredient10', 'strIngredient11', 'strIngredient12',
       'strIngredient13', 'strIngredient14', 'strIngredient15',
       'strIngredient2', 'strIngredient3', 'strIngredient4', 'strIngredient5',
       'strIngredient6', 'strIngredient7', 'strIngredient8', 'strIngredient9',
       'strInstructions', 'strMeasure1', 'strMeasure10', 'strMeasure11',
       'strMeasure12', 'strMeasure13', 'strMeasure14', 'strMeasure15',
       'strMeasure2', 'strMeasure3', 'strMeasure4', 'strMeasure5',
       'strMeasure6', 'strMeasure7', 'strMeasure8', 'strMeasure9', 'strVideo',
       'id', 'glass'],
      dtype='object')

In [8]:
df = df.rename(columns={'id': 'id_glass'})

In [9]:
df.columns

Index(['Unnamed: 0', 'strDrink', 'dateModified', 'idDrink', 'strAlcoholic',
       'strCategory', 'strDrinkThumb', 'strGlass', 'strIBA', 'strIngredient1',
       'strIngredient10', 'strIngredient11', 'strIngredient12',
       'strIngredient13', 'strIngredient14', 'strIngredient15',
       'strIngredient2', 'strIngredient3', 'strIngredient4', 'strIngredient5',
       'strIngredient6', 'strIngredient7', 'strIngredient8', 'strIngredient9',
       'strInstructions', 'strMeasure1', 'strMeasure10', 'strMeasure11',
       'strMeasure12', 'strMeasure13', 'strMeasure14', 'strMeasure15',
       'strMeasure2', 'strMeasure3', 'strMeasure4', 'strMeasure5',
       'strMeasure6', 'strMeasure7', 'strMeasure8', 'strMeasure9', 'strVideo',
       'id_glass', 'glass'],
      dtype='object')

In [10]:
df_categories = pd.read_sql_query("SELECT * FROM category", conn)

In [11]:
df = pd.merge(df, df_categories, left_on='strCategory', right_on='category')

In [12]:
df = df.rename(columns={'id': 'id_category'})

In [13]:
df = df.sort_values(by = 'strDrink')

In [14]:
df_temp = pd.DataFrame()

In [15]:
df_temp = df[['strDrink', 'strInstructions', 'id_glass', 'id_category']]

In [16]:
df_temp = df_temp.rename(columns={'strDrink': 'cocktail', 'strInstructions': 'instruction'})

In [17]:
ids = range(1, len(df_temp) + 1)

In [20]:
data_dict = {'cocktail': df_temp['cocktail'], 'instruction': df_temp['instruction'],
             'id_glass': df_temp['id_glass'],'id_category': df_temp['id_category']}
df_cocktail = pd.DataFrame(data_dict)

In [21]:
cursor = conn.cursor()

In [22]:
cursor.execute('''
    CREATE TABLE cocktail (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        cocktail VARCHAR(255),
        instruction TEXT,
        id_glass INTEGER,
        id_category INTEGER,
        FOREIGN KEY ("id_glass") REFERENCES glass (id),
        FOREIGN KEY ("id_category") REFERENCES category (id)
    )
''')

In [23]:
conn.commit()

In [24]:
df_cocktail.to_sql('cocktail', conn, index=False, if_exists='append')

546

In [25]:
conn.commit()

In [26]:
conn.close()